In [1]:
from mpl_toolkits.mplot3d import Axes3D
from rtree import index
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [2]:
%store -r mc_data

In [3]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        xlayer_model = load_model('./models/xlayer_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
###### Data columns
i = 0
for val in mc_data.columns:
    print(i," ",val)
    i = i + 1

0   MCParticleID
1   MC_Hit_X
2   MC_Hit_Y
3   MC_Hit_Z
4   PrHit_LHCbID
5   PrHit_Xat0
6   PrHit_Zat0
7   PrHit_isX
8   PrHit_planeCode
9   PrHit_w2
10   PrHit_yMax
11   PrHit_yMin
12   PrHit_zone
13   event


In [5]:
mc_hits = mc_data.values

In [6]:
%store -r zLays
%store -r max_x
%store -r min_x
%store -r events

In [7]:
print(events)

[15837 15838 15839 15840 15841 15842 15843 15844 15845 15846 15847 15848
 15849 15850 15851 15852 15853 15854 15855 15856 15857 15858 15859 15860
 15861 15862 15863 15864 15865 15866 15867 15868 15869 15870 15871 15872
 15873 15874 15875 15876 15877 15878 15879 15880 15881 15882 15883 15884
 15885 15886 15887 15888 15889 15890 15891 15892 15893 15894 15895 15896
 15897 15898 15899 15900 15901 15902 15903 15904 15905 15906 15907 15908
 15909 15910 15911 15912 15913 15914 15915 15916 15917 15918 15919 15920
 15921 15922 15923 15924 15925 15926 15927 15928 15929 15930 15931 15932
 15933 15934 15935 15936 15937 15938 15939 15940 15941 17909 17910 17911
 17912 17913 17914 17915 17916 17917 17918 17919 17920 17921 17922 17923
 17924 17925 17926 17927 17928 17929 17930 17931 17932 17933 17934 17935
 17936 17937 17938 17939 17940 17941 17942 17943 17944 17945 17946 17947
 17948 17949 17950 17951 17952 17953 17954 17955 17956 17957 17958 17959
 17960 17961 17962 17963 17964 17965 17966 17967 17

In [8]:
event_threshold = 15840

In [17]:
hits_total = mc_hits[mc_hits[:,13] < event_threshold]
print("Total hits with threshold %d" % (len(hits_total)))
event = 0
idx = {}
actual = 0
for hit in hits_total:
    if (not hit[13] in idx):
        p = index.Property()
        p.dimension = 3
        idx[hit[13]] = index.Index(properties=p, interleaved=False)
    
    if (len(hit[5]) > 1):
        for i in range(0,len(hit[5])):
            idx[hit[13]].insert(int(hit[4][i]), (float(hit[5][i]), float(hit[5][i]), hit[8][i], hit[8][i], hit[12][i], hit[12][i]), float(hit[5][i]))

Total hits with threshold 631


In [10]:
X_classifier = []
Y_classifier = []

total_found = 0
total = 0
long_range = 300

for particle in hits_total:
    
    if (len(particle[5]) > 1):
        
        found_bool = 0
        found_layers = np.zeros(4)
        # Extract only 12 hits, as some of them are really close
        zone = particle[12][0]
        X_final = {}
        actual = -1
        
        for i in range(0, len(particle[5])):
            if (actual != particle[8][i]):
                X_final[particle[8][i]] = []
                X_final[particle[8][i]].append(particle[5][i])
                actual = particle[8][i]
            else:
                X_final[actual].append(particle[5][i])
                
        if (not 0 in X_final):
            continue
            
        found_layers[0] = X_final[0][0]
            
        predicted = (xlayer_model.predict(np.array([X_final[0][0], 0]).reshape(1,-1)))[0]
        
        if (3 in X_final):
            found = list(idx[particle[13]].intersection((predicted[2] - long_range, predicted[2] + long_range, 3, 3, zone, zone),objects='raw'))
            found_layer3 = sorted(found, key=lambda x: np.abs(x - predicted[2]))[:20]

            for i in range(0,len(found_layer3)):
                if (found_layer3[i] in X_final[3]):
                    found_bool += 1
                    found_layers[3] = found_layer3[i]
                    break
            
        if (2 in X_final):
            found = list(idx[particle[13]].intersection((predicted[1] - long_range, predicted[1] + long_range, 2, 2, zone,zone),objects='raw'))
            found_layer2 = sorted(found, key=lambda x: np.abs(x - predicted[1]))[:20]

            for i in range(0,len(found_layer2)):
                if (found_layer2[i] in X_final[2]):
                    found_bool += 1
                    found_layers[2] = found_layer2[i]
                    break
        
        if (1 in X_final):
            found = list(idx[particle[13]].intersection((predicted[0] - long_range, predicted[0] + long_range, 1, 1,zone,zone),objects='raw'))
            found_layer1 = sorted(found, key=lambda x: np.abs(x - predicted[0]))[:20]

            for i in range(0,len(found_layer1)):
                if (found_layer1[i] in X_final[1]):
                    found_bool += 1
                    found_layers[1] = found_layer1[i]
                    break
            
        if (found_bool == 3):
            fake_track = np.array([X_final[0][0], random.choice(found_layer1),random.choice(found_layer2),random.choice(found_layer3)])
            true_track = found_layers
            
            retries = 0
            
            while (fake_track == true_track).all() and retries != 4:
                fake_track = np.array([X_final[0][0], random.choice(found_layer1),random.choice(found_layer2),random.choice(found_layer3)])
                retries += 1
                
            total_found += 1
            
            if (retries == 4):
                X_classifier.append(true_track)
                Y_classifier.append(1)
            else:
                X_classifier.append(true_track)
                Y_classifier.append(1)
                X_classifier.append(fake_track)
                Y_classifier.append(0)
            
        total += 1

print("Mean found %f" % (total_found/total))
print("Total %d" % total)

Mean found 0.856793
Total 126125


In [11]:
Y_classifier.count(1)

108063

In [12]:
track_classifier = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
track_classifier.add(Dense(12, activation='relu', input_dim=4))
track_classifier.add(Dense(8, activation='relu'))
track_classifier.add(Dense(1, activation='sigmoid'))
track_classifier.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_classifier), np.array(Y_classifier))
print("X Train length: %d" % len(X_train))
print("X Test Length %d" % len(X_test))
print("Y Train length: %d" % len(y_train))
print("Y Test length: %d" % len(y_test))

X Train length: 161922
X Test Length 53974
Y Train length: 161922
Y Test length: 53974


In [14]:
track_classifier.fit(X_train, y_train, epochs=10, batch_size=200, verbose=1)

Epoch 1/10
161922/161922 [==============================] - 2s 14us/sample - loss: 1.7536 - acc: 0.7997
Epoch 2/10
161922/161922 [==============================] - 2s 13us/sample - loss: 0.2828 - acc: 0.9119
Epoch 3/10
161922/161922 [==============================] - 2s 13us/sample - loss: 0.2268 - acc: 0.9281
Epoch 4/10
161922/161922 [==============================] - 2s 13us/sample - loss: 0.2210 - acc: 0.9354
Epoch 5/10
161922/161922 [==============================] - 2s 13us/sample - loss: 0.1864 - acc: 0.9431
Epoch 6/10
161922/161922 [==============================] - 2s 13us/sample - loss: 0.1853 - acc: 0.9448
Epoch 7/10
161922/161922 [==============================] - 2s 13us/sample - loss: 0.1852 - acc: 0.9460
Epoch 8/10
161922/161922 [==============================] - 2s 13us/sample - loss: 0.1915 - acc: 0.9456
Epoch 9/10
161922/161922 [==============================] - 2s 13us/sample - loss: 0.1794 - acc: 0.9484
Epoch 10/10
161922/161922 [==============================] - 2s 

In [15]:
score, acc = track_classifier.evaluate(X_test, y_test, batch_size=25)
print('Test score:', score)
print('Test accuracy:', acc)

53974/53974 [==============================] - 2s 43us/sample - loss: 0.1333 - acc: 0.9594
Test score: 0.13332966707968516
Test accuracy: 0.9594064


In [28]:
track_classifier.save("./models/track_classifier.h5")

In [38]:
import datetime as dt
X_test = []

total_found = 0
total = 0
long_range = 300

hits_total = mc_hits[mc_hits[:,13] < event_threshold]

for particle in hits_total:
    
    if (len(particle[5]) > 1):
        
        found_bool = 0
        found_layers = np.zeros(3)
        # Extract only 12 hits, as some of them are really close
        zone = particle[12][0]
        X_final = {}
        actual = -1
        
        for i in range(0, len(particle[5])):
            if (actual != particle[8][i]):
                X_final[particle[8][i]] = []
                X_final[particle[8][i]].append(particle[5][i])
                actual = particle[8][i]
            else:
                X_final[actual].append(particle[5][i])
                
        if (not 0 in X_final):
            continue
            
        predicted = (xlayer_model.predict(np.array([X_final[0][0], 0]).reshape(1,-1)))[0]
        
        if (3 in X_final):
            found = list(idx[particle[13]].intersection((predicted[2] - long_range, predicted[2] + long_range, 3, 3, zone, zone),objects='raw'))
            found_layer3 = sorted(found, key=lambda x: np.abs(x - predicted[2]))[:10]
            found_bool += 1
            
        if (2 in X_final):
            found = list(idx[particle[13]].intersection((predicted[1] - long_range, predicted[1] + long_range, 2, 2, zone,zone),objects='raw'))
            found_layer2 = sorted(found, key=lambda x: np.abs(x - predicted[1]))[:10]
            found_bool += 1
        
        if (1 in X_final):
            found = list(idx[particle[13]].intersection((predicted[0] - long_range, predicted[0] + long_range, 1, 1,zone,zone),objects='raw'))
            found_layer1 = sorted(found, key=lambda x: np.abs(x - predicted[0]))[:10]
            found_bool += 1
            
        if (found_bool == 3):
            retries = 0
            
            valid_list = []
            max_val = 0
            for h3 in found_layer3[:8]:
                for h2 in found_layer2[:8]:
                    for h1 in found_layer1[:8]:
                        pred = (track_classifier.predict(np.array([X_final[0][0], h1, h2, h3]).reshape(1,-1))[0])[0]
                        if pred > max_val:
                            valid = [X_final[0][0], h1, h2, h3]
                            max_val = pred
            
            total_val = 0
            if valid[1] in X_final[1]:
                total_val = 1
                
            if valid[2] in X_final[2]:
                total_val = 1
                
            if valid[3] in X_final[3]:
                total_val = 1
                
            total_found += total_val
                
            print(total/354)
            print(total_found/354)
            #print(total_found/354)
            #print(total_val)
            #print('\n')
            
        total += 1

print("Mean found %f" % (total_found/total))
print("Total %d" % total)

0.0
0.002824858757062147
0.002824858757062147
0.005649717514124294
0.005649717514124294
0.00847457627118644
0.00847457627118644
0.00847457627118644
0.011299435028248588
0.00847457627118644
0.014124293785310734
0.011299435028248588
0.01694915254237288
0.014124293785310734
0.01977401129943503
0.014124293785310734
0.022598870056497175
0.01694915254237288
0.025423728813559324
0.01977401129943503
0.02824858757062147
0.022598870056497175
0.031073446327683617
0.025423728813559324
0.03389830508474576
0.02824858757062147
0.03672316384180791
0.031073446327683617
0.03954802259887006
0.031073446327683617
0.0423728813559322
0.031073446327683617
0.04519774011299435
0.03389830508474576
0.0480225988700565
0.03672316384180791
0.05084745762711865
0.03954802259887006
0.05367231638418079
0.0423728813559322
0.05649717514124294
0.0423728813559322
0.059322033898305086
0.04519774011299435
0.062146892655367235
0.0480225988700565
0.06779661016949153
0.05084745762711865
0.07062146892655367
0.05367231638418079
0.

0.6412429378531074
0.4124293785310734
0.6440677966101694
0.4124293785310734
0.6468926553672316
0.4152542372881356
0.6497175141242938
0.4180790960451977
0.652542372881356
0.4180790960451977
0.6581920903954802
0.4180790960451977
0.6638418079096046
0.4209039548022599
0.6666666666666666
0.4209039548022599
0.6694915254237288
0.423728813559322
0.672316384180791
0.4265536723163842
0.6751412429378532
0.4293785310734463
0.6779661016949152
0.4322033898305085
0.6807909604519774
0.4322033898305085
0.6836158192090396
0.4350282485875706
0.6864406779661016
0.4350282485875706
0.6892655367231638
0.4378531073446328
0.692090395480226
0.4406779661016949
0.6949152542372882
0.4435028248587571
0.6977401129943502
0.4463276836158192
0.7005649717514124
0.4491525423728814
0.7033898305084746
0.4519774011299435
0.7062146892655368
0.4519774011299435
0.7090395480225988
0.4548022598870056
0.711864406779661
0.4548022598870056
0.7146892655367232
0.4548022598870056
0.7175141242937854
0.4548022598870056
0.723163841807909